In [ ]:
import os 
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from agent_framework import Executor, WorkflowBuilder, WorkflowContext,WorkflowOutputEvent , handler
from agent_framework import WorkflowBuilder ,WorkflowViz
from agent_framework import ChatAgent 
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import DefaultAzureCredential,AzureCliCredential

In [2]:
load_dotenv()

project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")


In [ ]:
async def create_agent(agent_name : str, agent_instructions : str) -> ChatAgent :
    credential = AzAzureCliCredential()
    chat_client = AzureAIAgentclient(
        async_credential=credential,
        project_endpoint=project_endpoint,
        model_deployment_name = model
    )
    agent = chat_client.create_agent(
        name = agent_name,
        instructions = agent_instructions
    )
    return agent

In [ ]:
location_picker = await create_agent(
    agent_name= "Destination picker agent",
    agent_instructions= "You are an expert travel agent. Your task is to help users
)

In [ ]:
destination_recommender = await create_agent(
    agent_name= "Destination recommender agent",
    agent_instructions= "You are an expert travel agent. Your task is to recommend travel destinations based on user preferences."
)


In [ ]:
weather_agent = await create_agent(
    agent_name= "Weather agent",
    agent_instructions= "You are an expert weather agent. Your task is to provide accurate and up-to-date weather information for various locations."
)

In [ ]:
cuisine_agent = await create_agent(
    agent_name= "Cuisine agent",
    agent_instructions= "You are an expert cuisine agent. Your task is to suggest popular cuisines and dishes from various destinations."
)

In [ ]:
itinerary_planner_agent = await create_agent(
    agent_name= "Itinerary planner agent",
    agent_instructions= "You are an expert itinerary planner. Your task is to create detailed travel itineraries based on user preferences and inputs from other agents."
)

In [ ]:
class LocationSelectorExecutor(Executor):
    @handler
    async def handler(self, user_query : str , ctx : WorkflowContext[str]) -> None :
        response = await location_picker.chat(user_query)
        ctx.set_output(response.content)

In [ ]:
class DestinationRecommenderExecutor(Executor):

    @handler
    async def handle(self, location: str, ctx: WorkflowContext[str]) -> None:
        response = await destination_recommender_agent.run(location)

        await ctx.send_message(str(response))

In [ ]:

class WeatherExecutor(Executor):

    @handler
    async def handle(self, location: str, ctx: WorkflowContext[str]) -> None:
        response = await weather_agent.run(location)

        await ctx.send_message(str(response))

In [ ]:

class CuisineSuggestionExecutor(Executor):

    @handler
    async def handle(self, location: str, ctx: WorkflowContext[str]) -> None:
        response = await cuisine_suggestion_agent.run(location)

        await ctx.send_message(str(response))

In [ ]:
class ItineraryPlannerExecutor(Executor):

    @handler
    async def handle(self, results: list[str], ctx: WorkflowContext[str]) -> None:
        response = await itinerary_planner_agent.run(results)

        await ctx.yield_output(str(response))

In [ ]:
location_selector_executor = LocationSelectorExecutor(id="LocationSelector")
destination_recommender_executor = DestinationRecommenderExecutor(id="DestinationRecommender")
weather_executor = WeatherExecutor(id="Weather")
cuisine_suggestion_executor = CuisineSuggestionExecutor(id="CuisineSuggestion")
itinerary_planner_executor = ItineraryPlannerExecutor(id="ItineraryPlanner")

# Build the workflow
workflow = (
    WorkflowBuilder()
    .set_start_executor(location_selector_executor)
    .add_fan_out_edges(location_selector_executor, [destination_recommender_executor, weather_executor, cuisine_suggestion_executor])
    .add_fan_in_edges([destination_recommender_executor, weather_executor, cuisine_suggestion_executor], itinerary_planner_executor)
    .build()
)

viz = WorkflowViz(workflow)

In [ ]:
mermaid_content = viz.to_mermaid()


In [ ]:
async def main():
    async for event in workflow.run_stream("help me plan a vacation to India with the following details: I love historical sites, prefer warm weather, and enjoy trying local foods."):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            print(f"Workflow completed with result: {event.data}")

await main()